<a href="https://colab.research.google.com/github/LucianoRodriguez0764/ipadapter/blob/main/Ipadapter_faceID_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies and download the IP-Adapter model

In [ ]:
!pip install insightface
!pip install onnxruntime
!pip install diffusers
!pip install git+https://github.com/tencent-ailab/IP-Adapter.git
!pip install einops

In [ ]:
!wget -O ip-adapter-faceid_sd15.bin https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin?download=true

# Create the pipe, vae and noise scheduler for IP-Adapter

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL, AutoPipelineForText2Image
from PIL import Image

from ip_adapter.ip_adapter_faceid import IPAdapterFaceID

base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
#new model
base_model_path = "SG161222/Realistic_Vision_V6.0_B1_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
ip_ckpt = "ip-adapter-faceid_sd15.bin"
device = "cuda"

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)

pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)




#Loading face analysis app and face embeddings

In [ ]:
import torch
import cv2
from insightface.app import FaceAnalysis


app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

In [ ]:
import os
import glob
import cv2

# Set the directory containing images
image_path = 'images/face'

imgs = glob.glob(os.path.join(image_path, '*.*'))
#imgs += glob.glob(os.path.join('images/face_2', '*.*'))
#imgs += glob.glob(os.path.join('images/face_3', '*.*'))

# Collect all embeddings
embeddings = []

for i in range(len(imgs)):
    print(imgs[i])
    image = cv2.imread(imgs[i])
    faces = app.get(image)

    # Check if a face is detected
    if faces:
        faceid_embed = torch.from_numpy(faces[0].normed_embedding).unsqueeze(0)
        embeddings.append(faceid_embed)
    else:
        print("face not detected")

# Calculate the average embedding for the face ID

weights = []
weights = [elem/sum(weights) for elem in weights]
#print(weights)

if embeddings:

    ### weights
    total = torch.zeros_like(embeddings[0])
    for i in range(len(embeddings)):
        if len(weights)==len(embeddings):
            total.add_(embeddings[i]*weights[i])
        else:
            total.add_(embeddings[i]*(1/len(embeddings)))
    faceid_embeds = total / len(embeddings[i])

    ###  mean weights
    #faceid_embeds = torch.mean(torch.stack(embeddings), dim=0)

    print("Collected face ID embedding")
else:
    print("No faces detected in the images provided.")


# Generate the image with IP-Adapter

In [ ]:
from IPython.display import display
from ipywidgets import HBox, Image
import random

def display_images(images, format="png"):
  # Assuming images is a list of PIL Images
  if format=="png" or True:
    image_widgets = [Image(value=img._repr_png_()) for img in images]
  # Display the images in a row
  display(HBox(image_widgets))

# load ip-adapter
ip_model = IPAdapterFaceID(pipe, ip_ckpt, device)
ip_model.set_scale(1)

prompt = ""
prompt += "photo of a 20 years old girl"
extra_prompt = ""

negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime), text, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, nfsw, extra legs, fused fingers, too many fingers, long neck"
negative_prompt += ""

width = 512
height = 512
num_inference_steps = 50
guidance_scale = 4
num_samples = 1
random_seed = 7030977
use_hardcoded_seed = False


x_plot_generation = True
y_plot_generation = True
x_plot_parameter = 'num_inference_steps'
x_plot_values = [30,50,70]
y_plot_parameter = 'extra_prompt'
y_plot_values = ["wearing blue and cyan dress, bun hair","wearing black and gold dress, ponytail hair"]
xy_different_seeds = False

if not use_hardcoded_seed:
  random_seed = random.randint(1, 9999999)

if x_plot_generation:
  num_samples=1
  if y_plot_generation:
    images_array = []
    for h in range(len(y_plot_values)):
      images = []
      if y_plot_parameter in globals():
        globals()[y_plot_parameter] = y_plot_values[h]
        print(y_plot_parameter,"set to:",y_plot_values[h])
      for i in range(len(x_plot_values)):
        if x_plot_parameter in globals():
          globals()[x_plot_parameter] = x_plot_values[i]
          print(x_plot_parameter,"set to:",x_plot_values[i])
        if xy_different_seeds and i!=0:
          random_seed = random.randint(1, 9999999)
        print("Seed: ", random_seed)
        generated_image = ip_model.generate(
            prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, faceid_embeds=faceid_embeds, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
            guidance_scale=guidance_scale
        )[0]
        images.append(generated_image)
      images_array.append(images)
    for img in images_array:
      display_images(img)
  elif x_plot_generation:
    images = []
    for i in range(len(x_plot_values)):
      if x_plot_parameter in globals():
        globals()[x_plot_parameter] = x_plot_values[i]
        print(x_plot_parameter,"set to:",x_plot_values[i])
      if xy_different_seeds and i!=0:
        random_seed = random.randint(1, 9999999)
      print("Seed: ", random_seed)
      images.extend(ip_model.generate(
          prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, faceid_embeds=faceid_embeds, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
          guidance_scale=guidance_scale
      ))
    display_images(images)


else:

  print("Seed: ", random_seed)
  images = ip_model.generate(
      prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, faceid_embeds=faceid_embeds, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
      guidance_scale=guidance_scale
  )

  display_images(images)

